In [2]:
import numpy as np
from helpers import make_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPool2D,
    BatchNormalization,
    Activation,
    Reshape,
    Flatten,
    Dense,
)

def gen_model():
    IMAGE_SIZE = 200
    NFILTERS = 8
    CONV_PARAMS = {"kernel_size": 3, "use_bias": False, "padding": "same"}

    model = Sequential()
    model.add(
        Reshape((IMAGE_SIZE, IMAGE_SIZE, 1), input_shape=(IMAGE_SIZE, IMAGE_SIZE))
    )
    for i in [1, 2, 4, 8, 16, 32, 64]:
        model.add(Conv2D(NFILTERS * i, **CONV_PARAMS))
        model.add(BatchNormalization())
        model.add(Activation("relu"))
        model.add(MaxPool2D())
    model.add(Flatten())
    model.add(Dense(5))
    return model


def make_batch(batch_size):
    # this model can only train on data where a spaceship is guaranteed, this is not true when testing
    imgs, labels = zip(*[make_data(has_spaceship=True) for _ in range(batch_size)])
    imgs = np.stack(imgs)
    labels = np.stack(labels)
    return imgs, labels


def main():
    BATCH_SIZE = 64

    model = gen_model()
    model.compile(loss="mse", optimizer="adam")
    model.summary()

    model.fit_generator(
        iter(lambda: make_batch(BATCH_SIZE), None), steps_per_epoch=500, epochs=30,
    )
    model.save("model.hdf5")
    print("Done")

In [3]:
main()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 200, 200, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 200, 200, 8)       72        
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200, 8)       32        
_________________________________________________________________
activation (Activation)      (None, 200, 200, 8)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 16)      1152      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 16)      6

In [4]:
from helpers import make_data, score_iou
import numpy as np
from tensorflow import keras
from tqdm import tqdm


def eval():
    model = keras.models.load_model("model.hdf5")

    ious = []
    for _ in tqdm(range(1000)):
        img, label = make_data()
        pred = model.predict(img[None])
        pred = np.squeeze(pred)
        ious.append(score_iou(label, pred))

    ious = np.asarray(ious, dtype="float")
    ious = ious[~np.isnan(ious)]  # remove true negatives
    print()
    print((ious > 0.7).mean())
    # 0.004

In [5]:
eval()

100%|██████████| 1000/1000 [00:37<00:00, 26.74it/s]


0.025


Model trained for 30 epochs had eval score of 0.04 or 0.025

Model trained for 2 epochs had eval score of 0.004